## State

- State : A Shared Data structure it can be created by using `TypesDict`  or Pydantic `BaseModel`. When we define the graph first things we do is defining the graph of the state they represent the current snapshot of the application. By default the graph will have same input & output schema. but we can change it see this  [notebook here](https://langchain-ai.github.io/langgraph/how-tos/input_output_schema/).
    - Schema : The main documented way to define the schema of a graph using `TypedDict`. the graph will have same input and output schema. If you want to change this, you can also explicitly specify this. this is use full when you have lots of keys.

- MultiSchema :  Typically, All nodes communicate with single schema. this means they will read and write to the same state channels. but there are cases when we want more control over this.
    - Internal nodes can pass information that is not required in the graph's input or output.
    - We may also want to use different input / output schemas for the graph. The output might, for example, only contain a single relevant output key.
        
    It is possible to have nodes with private state channel inside the graph for internal nodes communication. We can simply define a private Schema. See  [this notebook](https://langchain-ai.github.io/langgraph/how-tos/pass_private_state/).
        
    It is also possible to define explicitly input or output schema of graph. In these cases, we define `internal` schema that contains all keys relevant to graph operation. we also define `input` and `output` schemas that are sub-sets of the "internal" schema to constrain the input and output of the graph.

In [ ]:
from langgraph.graph import StateGraph, START,END
from typing_extensions import TypedDict

class InputState(TypedDict):
    question:str

class OutputState(TypedDict):
    answer:str
    masala:bool

class OverallState(InputState,OutputState):
    pass

# Define the node that process the input and generate an answer
def answer_node(state:InputState):
    return {"answer":"bye","question":state["question"]}

builder = StateGraph(OverallState,input = InputState, output = OutputState)
builder.add_node(answer_node)
builder.add_node(add_masala)
builder.add_edge(START, "answer_node")
builder.add_edge("answer_node", END)

graph = builder.compile()

print(graph.invoke({"question": "hi"}))

{'question': 'hi'}
{'answer': 'bye', 'masala': True}
